<a href="https://colab.research.google.com/github/nimab16/Music-Generation-LSTM-GAN/blob/main/Classical_Piano_Composer_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clone the dataset from github

In [13]:
!git clone https://github.com/nimab16/Music-Generation-LSTM-GAN

fatal: destination path 'Music-Generation-LSTM-GAN' already exists and is not an empty directory.


# install libraries

In [2]:
!pip install music21
!pip install Keras
!pip install Tensorflow
!pip install h5py

     |████████████████████████████████| 462 kB 35.2 MB/s 


# Import the librarries for train and prepare sequences 
This module prepares midi file data and feeds it to the neural network for training

In [3]:

import os 
os.chdir('/content/Music-Generation-LSTM-GAN')
import os
import sys
# Root directory of the project
ROOT_DIR = os.path.abspath("")

# Import Mask RCNN
sys.path.append(ROOT_DIR)


In [4]:
import glob
import pickle
import numpy
from music21 import converter, instrument, note, chord
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import BatchNormalization as BatchNorm
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

# Get all the notes and chords
Get all the notes and chords from the midi files in the ./midi_songs directory

In [5]:
def get_notes():
    """  """
    notes = []

    for file in glob.glob("midi_songs/*.mid"):
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = None

        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse() 
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    with open('data/notes', 'wb') as filepath:
        pickle.dump(notes, filepath)

    return notes

# Prepare the sequences
Prepare the sequences used by the Neural Network

In [6]:

def prepare_sequences(notes, n_vocab):
   
    sequence_length = 100

    # get all pitch names
    pitchnames = sorted(set(item for item in notes))

     # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)

    network_output = np_utils.to_categorical(network_output)

    return (network_input, network_output)

# Create the structure of model
create the structure of the neural network

In [7]:
def create_network(network_input, n_vocab):
  
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
    model.add(LSTM(512))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    return model

# Train model with check point

In [8]:


def train(model, network_input, network_output):
    """ train the neural network """
    filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
    checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    callbacks_list = [checkpoint]

    model.fit(network_input, network_output, epochs=20, batch_size=128, callbacks=callbacks_list)


# Call all functions by order
Train a Neural Network to generate music

In [9]:
def train_network():
    
    notes = get_notes()

    # get amount of pitch names
    n_vocab = len(set(notes))

    network_input, network_output = prepare_sequences(notes, n_vocab)

    model = create_network(network_input, n_vocab)

    train(model, network_input, network_output)

# call train method to run the program

In [ ]:
train_network()

# Generate classic music based on the pretrained model 
I generate the piano music based on **weights.hdf5** 

In [11]:
!python Predict_Classical_Piano_Composer.ipynb

Traceback (most recent call last):
  File "Predict_Classical_Piano_Composer.ipynb", line 9, in <module>
    "include_colab_link": true
NameError: name 'true' is not defined
